출처: https://dacon.io/codeshare/3725?dtype=recent

In [22]:
import os
import glob
import json
from glob import glob
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

In [23]:
!ls

drive  sample_data


In [24]:
import os
path = './'
file_list = os.listdir(path)
print(file_list)

['.config', 'drive', 'sample_data']


In [25]:
file_list

['.config', 'drive', 'sample_data']

In [28]:
for i in train_files:
    print(i)

['/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_361.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_374.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_456.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_501.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_510.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_527.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_556.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_564.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation/PP/PP_050_609.json', '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/a

In [38]:
# 이건 저의 개인적인 코랩이기 때문에 다르신 분들은 다시 변경하시기 바랍니다.
## train_PATH와 test_PATH만 변경하면 됩니다.
# json 파일로 만들어서 분석이 더 용이하다고 생각되어 만든 것이니 더 좋은 의견 있으면 말씀해주시기 바랍니다.

## Train
train_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Train/annotation'
file_list = os.listdir(train_PATH)

train_files = []
for file in tqdm(file_list):
    dir = train_PATH + '/' + file
    json_list = glob(dir + '/' +'*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

## Test
test_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/Test/annotations'
file_list = os.listdir(test_PATH)

test_files = []
for file in tqdm(file_list):
    dir = test_PATH + '/' + file
    json_list = glob(dir + '/' + '*.json')
    train_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)

100%|██████████| 1000/1000 [00:00<00:00, 1072163.60it/s]

100%|██████████| 1000/1000 [00:00<00:00, 944875.87it/s]

100%|██████████| 1000/1000 [00:00<00:00, 933727.52it/s]

100%|██████████| 4/4 [00:01<00:00,  2.45it/s]
0it [00:00, ?it/s]


In [40]:
import os
import numpy as np
import torch
from PIL import Image

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms = None, mode = 'train', json_file = None):
        self.mode = mode
        self.root = root
        self.transforms = transforms
        self.json_file = json_file
        
    def __getitem__(self, idx):
        if self.mode == 'train':

            annot = json_file[idx]
            # Train의 경로로 이동한다.
            PATH = root + '/' + 'Train'

            with open(annot, 'r') as f:
                annot = json.loads(f.read())
                
            image_name = annot['images'][0]['file_name']
            dir = image_name[:2]

            PATH = PATH + '/' + dir + '/' + image_name
            
            image = Image.open(PATH).convert('RGB')

            boxes = []
            segmentations = []
            labels = []

            for i in range(len(annot['annotations'])):

                segmentation = annot['annotations'][i]['segmentation'][0]
                bbox = annot['annotations'][i]['bbox']
                label = annot['annotations'][i]['category_id']
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmin, ymin, xmax, ymax = xmin, ymin, xmin + width, ymax + height
                
                boxes.append([xmin, ymin, xmax, ymax])
                segmentations.append([segmentation])
                labels.append(label)

            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            target['segmentation'] = segmentations
            target['image_id'] = image_name

            if self.transforms is not None:
                img, target = self.transforms(img, target)

            return img, target

        else:
            # Test의 경로로 이동한다.
            PATH = root + '/' + 'Test'

            ##### 이 부분은 추후에 작성하기로 하자.
            # img_path = os.path
        
    def __len__(self):
        return len(self.json_file)

In [41]:
train_dataset = CustomDataset(train_json_list, mode = 'train')

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

num_classes = 4
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)